In [1]:
import utils_data as ut
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizer
tokenizerBERT = BertTokenizer.from_pretrained('pranav-s/MaterialsBERT', model_max_length=512)
modelBERT = BertForMaskedLM.from_pretrained('pranav-s/MaterialsBERT')

In [2]:
classes = {'POLYMER': 1,
           'ORGANIC': 2,
           'MONOMER': 3,
           'PROP_NAME': 4,
           'INORGANIC': 5,
           'MATERIAL_AMOUNT': 6,
           'POLYMER_FAMILY': 7,
           'PROP_VALUE': 8,
           'O': 0}
max_length = 512
batch_size = 10
class NERBERTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = modelBERT.base_model
        self.linear = nn.Linear(768, len(classes) + 1)
        
    def forward(self, token):
        encoder_output= self.bert(token)
        linear_output = self.linear(encoder_output.last_hidden_state[0,:,:])
        class_output = F.softmax(linear_output, dim=1)
        return class_output

In [3]:
model = NERBERTModel()

In [4]:
data_list = ut.read_data('train.json')

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
epochs = 1
train_losses = []
for epoch in range(epochs):
    for data in data_list[:100]:
        token = ut.list2token(tokenizerBERT, data['words'], max_length)
        prediction = model(token)
        target = torch.tensor(ut.cat2digit(classes, data['ner'], max_length))
        loss = criterion(prediction, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_losses.append(loss)
    print(f'epoch: {epoch:2}  loss: {loss.item():10.8f}')

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])
torch.Size([512, 10]) torch.Size([512])


KeyboardInterrupt: 